# NEXUS Colab Worker 🚀
GPU бесплатно от Google

In [ ]:
!pip install requests -q
!apt-get install -qq nmap > /dev/null
!wget -q https://github.com/projectdiscovery/nuclei/releases/download/v3.1.0/nuclei_3.1.0_linux_amd64.zip && unzip -q -o nuclei_3.1.0_linux_amd64.zip
!wget -q https://github.com/projectdiscovery/subfinder/releases/download/v2.6.3/subfinder_2.6.3_linux_amd64.zip && unzip -q -o subfinder_2.6.3_linux_amd64.zip
!wget -q https://github.com/projectdiscovery/httpx/releases/download/v1.3.7/httpx_1.3.7_linux_amd64.zip && unzip -q -o httpx_1.3.7_linux_amd64.zip
!chmod +x nuclei subfinder httpx
print('Tools ready!')

In [ ]:
import subprocess, requests, json

TARGET = 'example.com'  # @param {type:"string"}
VPS = 'http://176.123.169.38:5000'

def full_recon(target):
    results = {'target': target}
    
    # Subdomains
    print('[1/3] Subfinder...')
    r = subprocess.run(['./subfinder', '-d', target, '-silent'], capture_output=True, text=True, timeout=120)
    results['subdomains'] = r.stdout.strip().split('\n')
    print(f'  Found {len(results["subdomains"])} subdomains')
    
    # HTTP probe
    print('[2/3] HTTPX...')
    with open('subs.txt', 'w') as f: f.write(r.stdout)
    r2 = subprocess.run(['./httpx', '-l', 'subs.txt', '-silent'], capture_output=True, text=True, timeout=180)
    results['alive'] = r2.stdout.strip().split('\n')
    print(f'  Alive: {len(results["alive"])}')
    
    # Nuclei
    print('[3/3] Nuclei...')
    r3 = subprocess.run(['./nuclei', '-l', 'subs.txt', '-silent', '-json', '-rate-limit', '50'],
                       capture_output=True, text=True, timeout=600)
    vulns = []
    for line in r3.stdout.split('\n'):
        if line.strip():
            try: vulns.append(json.loads(line))
            except: pass
    results['vulnerabilities'] = vulns
    print(f'  Vulns: {len(vulns)}')
    
    # Send to VPS
    requests.post(f'{VPS}/catch/colab-result', json=results, timeout=10)
    return results

full_recon(TARGET)